# Wrangle Zillow

### If you are finished with the exercises...here is what I want y'all to do next...this will help you prepare for the regression project :
- use mySQL to query the zillow database.
- you will need to use the properties_2017 and predictions_2017 tables.
- you will want to gather the following information (and figure out which columns are most likely to give you this information)...square feet of the house ("living square feet"), number of bedrooms, number of bathrooms, the assess value of the house by the tax appraisal district ('taxvaluedollarcnt'...this will be your target variable in the project), and 2-3 other variables you think will be useful (think about what you know about what drives home prices, and also the amount of non-null values available in selecting your extra variables.
- you will want to limit your data to properties that were sold in may, june, july or august (date is in predictions_2017 table).
- There are a ton of rows...so think about how you can limit your data set early to keep your query going! i.e. filter the dates!

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np 
import os
from env import host, user, password 
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split